In [4]:
import requests
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from fuzzywuzzy import process


# # List of raw URLs to the .pickle files
# urls = [
#     'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/distance_df.pickle',
#     'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/flights_df.pickle',
#     'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/flights_df_all_fields.pickle',
#     'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/geos_df.pickle',
#     'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/msa_df.pickle',
#     'https://github.com/mamaOcoder/highspeedrail/raw/88761104ba44f569e5fadde614eb5ed6a54900ba/data/pickled/tti_df.pickle'
# ]

# loaded_data = []

# for url in urls:
#     response = requests.get(url)
#     if response.status_code == 200:
#         # Load the content of the file
#         data = pickle.loads(response.content)
#         loaded_data.append(data)
#     else:
#         print(f"Failed to retrieve {url}")

In [49]:
# # Convert loaded data to DataFrames
# distance_df = pd.DataFrame(loaded_data[0])
# flights_df = pd.DataFrame(loaded_data[1])
# flights_df_all_fields = pd.DataFrame(loaded_data[2])
# geos_df = pd.DataFrame(loaded_data[3])
# msa_df = pd.DataFrame(loaded_data[4])
# tti_df = pd.DataFrame(loaded_data[5])

# Load data
city_pair_weights_pickle = '../data/pickled/city_pair_weights_df.pickle'
with open(city_pair_weights_pickle, 'rb') as file:
    city_pair_df = pickle.load(file)

city_pair_df['efficiency_score_norm'] = (city_pair_df['efficiency_score'] - city_pair_df['efficiency_score'].min()) / (city_pair_df['efficiency_score'].max() - city_pair_df['efficiency_score'].min())

msa_weights_pickle = '../data/pickled/msa_weights_df.pickle'
with open(msa_weights_pickle, 'rb') as file:
    msa_df = pickle.load(file)
msa_df['demand_score_norm'] = (msa_df['demand_score'] - msa_df['demand_score'].min()) / (msa_df['demand_score'].max() - msa_df['demand_score'].min())

In [41]:
# TTI data is already merged into msa_df

# # correct column name is 'MetroArea'
# city_column_name = 'MetroArea'

# def get_best_match(city, choices):
#     match, score = process.extractOne(city, choices)
#     return match if score > 80 else None  

# all_city_names = set(msa_df['MainCity']).union(set(distance_df['Origin']), set(distance_df['Destination']))

# tti_df['Matched_City'] = tti_df[city_column_name].apply(lambda x: get_best_match(x, all_city_names))
# tti_data = tti_df.set_index('Matched_City')['TravelTimeIndexValue'].to_dict()

# tti_data

In [50]:
# Merge msa_df into city_pair_df to get additional data for city pairs
city_pairs = city_pair_df.merge(msa_df[['MainCity', 'PopNorm', 'GDPNorm',	'TTINorm', 'demand_score_norm', 'proximity_score', 'cluster_id']],
                                  left_on='City1', right_on='MainCity', suffixes=('', '_City1'))
city_pairs = city_pairs.merge(msa_df[['MainCity', 'PopNorm', 'GDPNorm',	'TTINorm', 'demand_score_norm', 'proximity_score', 'cluster_id']],
                                  left_on='City2', right_on='MainCity', suffixes=('', '_City2'))

In [51]:
city_pairs

,City1,City2,drive_distance,drive_duration,flight_distance,flight_duration_ramp,flight_duration_air,flight_duration_total,num_passengers,total_seats,...,demand_score_norm,proximity_score,cluster_id,MainCity_City2,PopNorm_City2,GDPNorm_City2,TTINorm_City2,demand_score_norm_City2,proximity_score_City2,cluster_id_City2
0,"Abilene, TX","Akron, OH",1327.729086,1159.0,1169.268938,0.0,0.0,0.0,0.0,0.0,...,0.002988,0.430527,3.0,"Akron, OH",0.03294,0.019143,0.173913,0.009708,0.743781,1.0
1,"Abilene, TX","Albany, GA",982.946785,910.0,915.647527,0.0,0.0,0.0,0.0,0.0,...,0.002988,0.430527,3.0,"Albany, GA",0.0045,0.001897,0.065217,0.001124,0.596153,4.0
2,"Abilene, TX","Albany, NY",1812.063858,1578.5,1581.642374,0.0,0.0,0.0,0.0,0.0,...,0.002988,0.430527,3.0,"Albany, NY",0.043551,0.035689,0.173913,0.033797,0.583088,1.0
3,"Abilene, TX","Albany, OR",1912.769767,1746.5,1511.136717,0.0,0.0,0.0,0.0,0.0,...,0.002988,0.430527,3.0,"Albany, OR",0.003779,0.001339,0.043478,0.000000,0.430416,0.0
4,"Abilene, TX","Albuquerque, NM",487.537939,448.5,437.507433,0.0,0.0,0.0,0.0,0.0,...,0.002988,0.430527,3.0,"Albuquerque, NM",0.044457,0.023448,0.260870,0.056945,0.359896,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69373,"York, PA","Yuba City, CA",2692.714957,2360.0,2372.324885,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.791775,1.0,"Yuba City, CA",0.006463,0.002497,0.239130,0.000000,0.386060,0.0
69374,"York, PA","Yuma, AZ",2440.286406,2156.5,2155.951318,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.791775,1.0,"Yuma, AZ",0.007983,0.003087,0.108696,0.002639,0.305170,0.0
69375,"Youngstown, OH","Yuba City, CA",2419.119713,2112.5,2154.189369,0.0,0.0,0.0,0.0,0.0,...,0.000003,0.698477,1.0,"Yuba City, CA",0.006463,0.002497,0.239130,0.000000,0.386060,0.0
69376,"Youngstown, OH","Yuma, AZ",2227.498217,1954.0,1955.046047,0.0,0.0,0.0,0.0,0.0,...,0.000003,0.698477,1.0,"Yuma, AZ",0.007983,0.003087,0.108696,0.002639,0.305170,0.0


In [57]:
city_pairs['gaus_dist_weight'].max()

np.float64(0.9831201702191406)

In [106]:
# Define weights

# MSA unique contributions
population_weight = 0.3
gdp_weight = 0.2 
tti_weight = 0.1 
demand_weight = 0.4 

# Relationship contributions
dist_weight = 0.4 
efficiency_weight = 0.1
cp_demand_weight = 0.5

# Use cluster as a boost
cluster_weight = 0.15


def calculate_score(row):
    city1_contribution = (population_weight * row['PopNorm'] +
                          gdp_weight * row['GDPNorm'] +
                          tti_weight * row['TTINorm'] +
                          demand_weight * row['demand_score_norm'])# +
                          #proximity_weight * row['proximity_score'])
    city2_contribution = (population_weight * row['PopNorm_City2'] +
                          gdp_weight * row['GDPNorm_City2'] +
                          tti_weight * row['TTINorm_City2'] +
                          demand_weight * row['demand_score_norm_City2'])# +
                          #proximity_weight * row['proximity_score_City2'])

    pair_contribution = (dist_weight * row['gaus_dist_weight'] +
                         efficiency_weight * row['efficiency_score_norm'] +
                         cp_demand_weight * row['demand_weight'])

    # Boost the score if the city pair is in the same cluster
    same_cluster_boost = cluster_weight if row['cluster_id'] == row['cluster_id_City2'] else 0

    return ((1/4) * city1_contribution + 
            (1/4) * city2_contribution + 
            (1/2) * pair_contribution + 
            same_cluster_boost)

# Calculate total and normalized scores
city_pairs['Total_Score'] = city_pairs.apply(calculate_score, axis=1)
#city_pairs['Normalized_Score'] = (city_pairs['Total_Score'] / city_pairs['Total_Score'].max())
results = city_pairs.sort_values(by='Total_Score', ascending=False)
results[['City1','City2','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(50)

,City1,City2,drive_distance,PopNorm,PopNorm_City2,GDPNorm,GDPNorm_City2,TTINorm,TTINorm_City2,demand_score_norm,demand_score_norm_City2,proximity_score,proximity_score_City2,gaus_dist_weight,efficiency_score_norm,demand_weight,cluster_id,cluster_id_City2,Total_Score
57619,"Miami, FL","New York, NY",1285.911440,0.315077,1.0,0.222473,1.000000,0.652174,0.608696,0.739325,1.000000,0.363144,0.628190,5.188724e-206,0.745210,1.000000,4.0,1.0,0.652469
14632,"Boston, MA","New York, NY",215.741565,0.250056,1.0,0.263173,1.000000,0.391304,0.608696,0.351059,1.000000,0.512763,0.628190,1.752387e-03,1.000000,0.279927,1.0,1.0,0.587351
6389,"Atlanta, GA","Miami, FL",664.196511,0.321459,0.315077,0.241979,0.222473,0.456522,0.652174,0.912698,0.739325,0.691560,0.363144,1.216358e-28,0.715373,0.491214,4.0,4.0,0.572455
60973,"New York, NY","Washington, DC",226.807871,1.0,0.321341,1.000000,0.304358,0.608696,0.456522,1.000000,0.477669,0.628190,0.709058,8.659782e-03,0.602993,0.200080,1.0,1.0,0.570617
60887,"New York, NY","Portland, ME",314.228868,1.0,0.026146,1.000000,0.018773,0.608696,0.086957,1.000000,0.020633,0.628190,0.421050,7.694434e-01,0.088785,0.038204,1.0,1.0,0.565233
55441,"Los Angeles, CA","San Francisco, CA",382.824809,0.655397,0.231938,0.566786,0.336061,1.000000,0.956522,0.532577,0.393646,0.338033,0.416524,1.548366e-01,0.746769,0.323964,0.0,0.0,0.552525
17566,"Burlington, VT","New York, NY",299.630067,0.00874,1.0,0.006604,1.000000,0.173913,0.608696,0.014532,1.000000,0.430317,0.628190,6.890235e-01,0.092141,0.037963,1.0,1.0,0.548907
52920,"Las Vegas, NV","Los Angeles, CA",270.797521,0.117207,0.655397,0.072923,0.566786,0.391304,1.000000,0.518789,0.532577,0.317168,0.338033,2.707235e-01,0.925459,0.265186,0.0,0.0,0.546564
60900,"New York, NY","Richmond, VA",338.741643,1.0,0.066445,1.000000,0.049212,0.608696,0.130435,1.000000,0.055568,0.628190,0.631269,5.771865e-01,0.099153,0.054166,1.0,1.0,0.540415
16921,"Buffalo, NY","New York, NY",373.182063,0.056459,1.0,0.037712,1.000000,0.195652,0.608696,0.052468,1.000000,0.567788,0.628190,4.789599e-01,0.151671,0.079574,1.0,1.0,0.529744


In [109]:
results[['City1','City2','Total_Score']].head(25)

,City1,City2,Total_Score
57619,"Miami, FL","New York, NY",0.652469
14632,"Boston, MA","New York, NY",0.587351
6389,"Atlanta, GA","Miami, FL",0.572455
60973,"New York, NY","Washington, DC",0.570617
60887,"New York, NY","Portland, ME",0.565233
55441,"Los Angeles, CA","San Francisco, CA",0.552525
17566,"Burlington, VT","New York, NY",0.548907
52920,"Las Vegas, NV","Los Angeles, CA",0.546564
60900,"New York, NY","Richmond, VA",0.540415
16921,"Buffalo, NY","New York, NY",0.529744


In [117]:
# Define weights

# MSA unique contributions
region_population_weight = 0.2
region_gdp_weight = 0.2 
region_tti_weight = 0.1 
region_demand_weight = 0.5 

# Relationship contributions
region_dist_weight = 0.2 
region_efficiency_weight = 0.2
region_cp_demand_weight = 0.6


def calculate_regional_score(row):
    city1_contribution = (region_population_weight * row['PopNorm'] +
                          region_gdp_weight * row['GDPNorm'] +
                          region_tti_weight * row['TTINorm'] +
                          region_demand_weight * row['demand_score_norm'])# +
                          #proximity_weight * row['proximity_score'])
    city2_contribution = (region_population_weight * row['PopNorm_City2'] +
                          region_gdp_weight * row['GDPNorm_City2'] +
                          region_tti_weight * row['TTINorm_City2'] +
                          region_demand_weight * row['demand_score_norm_City2'])# +
                          #proximity_weight * row['proximity_score_City2'])

    pair_contribution = (region_dist_weight * row['gaus_dist_weight'] +
                         region_efficiency_weight * row['efficiency_score_norm'] +
                         region_cp_demand_weight * row['demand_weight'])


    return ((1/3) * city1_contribution + 
            (1/3) * city2_contribution + 
            (1/3) * pair_contribution)

region0 = city_pairs.loc[(city_pairs['cluster_id']==0) & (city_pairs['cluster_id_City2']==0)].copy()
region1 = city_pairs.loc[(city_pairs['cluster_id']==1) & (city_pairs['cluster_id_City2']==1)].copy()
region2 = city_pairs.loc[(city_pairs['cluster_id']==2) & (city_pairs['cluster_id_City2']==2)].copy()
region3 = city_pairs.loc[(city_pairs['cluster_id']==3) & (city_pairs['cluster_id_City2']==3)].copy()
region4 = city_pairs.loc[(city_pairs['cluster_id']==4) & (city_pairs['cluster_id_City2']==4)].copy()



region0['Regional_Score'] = region0.apply(calculate_regional_score, axis=1)
results0 = region0.sort_values(by='Regional_Score', ascending=False)
results0[['City1','City2','Regional_Score','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(5)
results0[['City1','City2','Regional_Score']].head(10)

,City1,City2,Regional_Score
55441,"Los Angeles, CA","San Francisco, CA",0.463833
52920,"Las Vegas, NV","Los Angeles, CA",0.448542
29589,"Denver, CO","Los Angeles, CA",0.404602
55406,"Los Angeles, CA","Phoenix, AZ",0.392027
55450,"Los Angeles, CA","Seattle, WA",0.353088
29646,"Denver, CO","Phoenix, AZ",0.348865
52977,"Las Vegas, NV","Phoenix, AZ",0.348189
55442,"Los Angeles, CA","San Jose, CA",0.338169
29681,"Denver, CO","San Francisco, CA",0.331796
29575,"Denver, CO","Las Vegas, NV",0.322907


In [118]:
region1['Regional_Score'] = region1.apply(calculate_regional_score, axis=1)
results1 = region1.sort_values(by='Regional_Score', ascending=False)
results1[['City1','City2','Regional_Score','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(5)
results1[['City1','City2','Regional_Score']].head(10)

,City1,City2,Regional_Score
14632,"Boston, MA","New York, NY",0.548827
60973,"New York, NY","Washington, DC",0.537625
60881,"New York, NY","Philadelphia, PA",0.429902
60883,"New York, NY","Pittsburgh, PA",0.409905
31572,"Durham, NC","New York, NY",0.406129
16921,"Buffalo, NY","New York, NY",0.399791
60900,"New York, NY","Richmond, VA",0.397532
8521,"Baltimore, MD","New York, NY",0.394755
60887,"New York, NY","Portland, ME",0.394478
60968,"New York, NY","Virginia Beach, VA",0.392187


In [119]:
region2['Regional_Score'] = region2.apply(calculate_regional_score, axis=1)
results2 = region2.sort_values(by='Regional_Score', ascending=False)
results2[['City1','City2','Regional_Score','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(5)
results2[['City1','City2','Regional_Score']].head(10)

,City1,City2,Regional_Score
22126,"Chicago, IL","Detroit, MI",0.373602
22258,"Chicago, IL","Minneapolis, MN",0.372956
22359,"Chicago, IL","St. Louis, MO",0.341534
22101,"Chicago, IL","Cincinnati, OH",0.322334
22240,"Chicago, IL","Louisville, KY",0.321587
22112,"Chicago, IL","Columbus, OH",0.318695
22125,"Chicago, IL","Des Moines, IA",0.289181
22207,"Chicago, IL","Kansas City, MO",0.288605
22191,"Chicago, IL","Indianapolis, IN",0.285801
22257,"Chicago, IL","Milwaukee, WI",0.277168


In [120]:
region3['Regional_Score'] = region3.apply(calculate_regional_score, axis=1)
results3 = region3.sort_values(by='Regional_Score', ascending=False)
results3[['City1','City2','Regional_Score','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(5)
results3[['City1','City2','Regional_Score']].head(10)

,City1,City2,Regional_Score
27253,"Dallas, TX","Houston, TX",0.476767
7662,"Austin, TX","Dallas, TX",0.358635
27395,"Dallas, TX","San Antonio, TX",0.348523
27301,"Dallas, TX","Little Rock, AR",0.312061
27341,"Dallas, TX","New Orleans, LA",0.301269
27322,"Dallas, TX","Midland, TX",0.293083
2292,"Alexandria, LA","Dallas, TX",0.287495
27328,"Dallas, TX","Monroe, LA",0.287322
27211,"Dallas, TX","Fayetteville, AR",0.284462
27307,"Dallas, TX","Lubbock, TX",0.281801


In [121]:
region4['Regional_Score'] = region4.apply(calculate_regional_score, axis=1)
results4 = region4.sort_values(by='Regional_Score', ascending=False)
results4[['City1','City2','Regional_Score','drive_distance','PopNorm','PopNorm_City2','GDPNorm','GDPNorm_City2','TTINorm','TTINorm_City2','demand_score_norm','demand_score_norm_City2','proximity_score','proximity_score_City2','gaus_dist_weight','efficiency_score_norm','demand_weight','cluster_id','cluster_id_City2','Total_Score']].head(5)
results4[['City1','City2','Regional_Score']].head(10)

,City1,City2,Regional_Score
6389,"Atlanta, GA","Miami, FL",0.531627
6422,"Atlanta, GA","Orlando, FL",0.406607
21019,"Charlotte, NC","Miami, FL",0.375754
6231,"Atlanta, GA","Charlotte, NC",0.367011
57629,"Miami, FL","Orlando, FL",0.355756
6502,"Atlanta, GA","Tampa, FL",0.344621
57709,"Miami, FL","Tampa, FL",0.326185
6409,"Atlanta, GA","Nashville, TN",0.312254
6333,"Atlanta, GA","Jacksonville, FL",0.305021
6229,"Atlanta, GA","Charleston, SC",0.303048


In [13]:
# # Define scoring functions
# def score_population(pop):
#     pop = int(pop.replace(',', '')) if isinstance(pop, str) else pop
#     return (pop / max(city_population.values())) * 100

# def score_distance(dist):
#     if dist <= 150:
#         return ((dist / 100) + 1) * 10
#     elif 150 < dist <= 300:
#         return 25
#     elif 300 < dist <= 350:
#         return (((500 - dist) / 100) + 0.5) * 10
#     elif 350 < dist <= 500:
#         return ((500 - dist) / 100) * 10
#     else:
#         return 0

# def score_gdp(gdp):
#     return (gdp / max(city_gdp.values())) * 100

# def score_congestion(ttis, city):
#     if city in ttis:
#         return (ttis[city] / max(ttis.values())) * 100
#     else:
#         return 0

In [14]:
# msa_merged_df = pd.merge(msa_df, geos_df, left_on='MainCity', right_on='MainCity')

# city_population = msa_merged_df.set_index('MainCity')['Population'].to_dict()
# city_gdp = msa_merged_df.set_index('MainCity')['GDP_thousands_dollars'].to_dict()

# # Calculate scores 
# city_pairs = distance_df[['Origin', 'Destination', 'Distance_miles']].copy()
# city_pairs['Population_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_population(city_population.get(x, 0)))
# city_pairs['Population_Score_Destination'] = city_pairs['Destination'].map(lambda x: score_population(city_population.get(x, 0)))
# city_pairs['Distance_Score'] = city_pairs['Distance_miles'].map(score_distance)
# city_pairs['GDP_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_gdp(city_gdp.get(x, 0)))
# city_pairs['GDP_Score_Destination'] = city_pairs['Destination'].map(lambda x: score_gdp(city_gdp.get(x, 0)))
# city_pairs['Congestion_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_congestion(tti_data, x))
# city_pairs['Congestion_Score_Destination'] = city_pairs['Destination'].map(lambda x: score_congestion(tti_data, x))

# # Calculate total and normalized scores
# city_pairs['Total_Score'] = city_pairs[['Population_Score_Origin', 'Population_Score_Destination', 'Distance_Score', 'GDP_Score_Origin', 'GDP_Score_Destination', 'Congestion_Score_Origin', 'Congestion_Score_Destination']].sum(axis=1)
# city_pairs['Normalized_Score'] = (city_pairs['Total_Score'] / city_pairs['Total_Score'].max()) * 100


In [15]:
# Rank city pairs by their normalized scores
top_50_city_pairs = city_pairs.nlargest(50, 'Normalized_Score')

# Display top 50 city pairs
print(top_50_city_pairs[['Origin', 'Destination', 'Distance_miles', 
                         'Population_Score_Origin', 'Population_Score_Destination', 
                         'Distance_Score', 'GDP_Score_Origin', 
                         'GDP_Score_Destination', 'Congestion_Score_Origin', 'Congestion_Score_Destination', 
                         'Total_Score', 'Normalized_Score']])

                   Origin        Destination  Distance_miles  \
25629         Chicago, IL       New York, NY      789.945845   
93337        New York, NY        Chicago, IL      789.789260   
96326        New York, NY     Washington, DC      228.846590   
141429     Washington, DC       New York, NY      224.769153   
17859          Boston, MA       New York, NY      216.211632   
93132        New York, NY         Boston, MA      215.271497   
60369         Houston, TX       New York, NY     1628.373542   
94237        New York, NY        Houston, TX     1626.629353   
6339          Atlanta, GA       New York, NY      864.373664   
92838        New York, NY        Atlanta, GA      873.701064   
24727         Chicago, IL        Houston, TX     1073.605435   
58577         Houston, TX        Chicago, IL     1082.857649   
95920        New York, NY        Seattle, WA     2851.558511   
125929        Seattle, WA       New York, NY     2836.339892   
10159       Baltimore, MD       New York

In [16]:
# Display the top 50 city pairs
print(top_50_city_pairs[['Origin', 'Destination',  
                         'Total_Score', 'Normalized_Score']])

                   Origin        Destination  Total_Score  Normalized_Score
25629         Chicago, IL       New York, NY   382.305203        100.000000
93337        New York, NY        Chicago, IL   382.305203        100.000000
96326        New York, NY     Washington, DC   380.467872         99.519407
141429     Washington, DC       New York, NY   380.467872         99.519407
17859          Boston, MA       New York, NY   367.025985         96.003398
93132        New York, NY         Boston, MA   367.025985         96.003398
60369         Houston, TX       New York, NY   361.862267         94.652718
94237        New York, NY        Houston, TX   361.862267         94.652718
6339          Atlanta, GA       New York, NY   349.250996         91.353974
92838        New York, NY        Atlanta, GA   349.250996         91.353974
24727         Chicago, IL        Houston, TX   344.167470         90.024271
58577         Houston, TX        Chicago, IL   344.167470         90.024271
95920       

In [17]:
for i, df in enumerate(dataframes, start=1):
    print(f"Dataset {i}:")
    print("First few rows:")
    print(df.head(), "\n")  
    print("Data Types:")
    print(df.dtypes, "\n") 
    print("Descriptive Statistics:")
    print(df.describe(), "\n")  
    print("Missing Values:")
    print(df.isnull().sum(), "\n") 
    print("-" * 50)


NameError: name 'dataframes' is not defined

In [ ]:
for i, df in enumerate(dataframes, start=1):
    print(f"Visualizing Dataset {i}:")
    # Histograms for all numerical features
    df.hist(figsize=(10, 8))
    plt.show()

    # Pairplot 
    if len(df.columns) <= 10:  
        sns.pairplot(df)
        plt.show()

    # Boxplots for outliers
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in num_cols:
        plt.figure(figsize=(8, 4))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
        plt.show()

    print("-" * 100)


In [ ]:
for i, df in enumerate(dataframes, start=1):
    numeric_df = df.select_dtypes(include=[np.number])
    if not numeric_df.empty:
        plt.figure(figsize=(10, 8))
        sns.heatmap(numeric_df.corr(), annot=True, fmt=".2f", cmap='coolwarm')
        plt.title(f"Correlation Matrix for Dataset {i}")
        plt.show()
    else:
        print(f"Dataset {i} contains no numeric columns suitable for correlation.")
